In [1]:
from glob import glob
import pandas as pd
import re
from pprint import pprint
import pkg_resources

from pymedextcore.document import Document
from pymedext_eds.annotators import Endlines, SentenceTokenizer, SectionSplitter, rawtext_loader
from pymedext_eds.med import MedicationAnnotator, MedicationNormalizer

In [2]:
models_param = [{'tagger_path':'/export/home/cse180025/prod_information_extraction/data/models/apmed4/entities/final-model.pt' ,
                'tag_name': 'entity_pred' },
                {'tagger_path':'/export/home/cse180025/prod_information_extraction/data/models/apmed4/events/final-model.pt' ,
                'tag_name': 'event_pred' },
               {'tagger_path': "/export/home/cse180025/prod_information_extraction/data/models/apmed4/drugblob/final-model.pt",
                'tag_name': 'drugblob_pred'}]

data_path = pkg_resources.resource_filename('pymedext_eds', 'data/romedi')
romedi_path = glob(data_path + '/*.p')[0]

In [3]:
endlines = Endlines(["raw_text"], "clean_text", ID="endlines")
sections = SectionSplitter(['clean_text'], "section", ID= 'sections')
sentenceSplitter = SentenceTokenizer(["section"],"sentence", ID="sentences")
med = MedicationAnnotator(['sentence'], 'med', ID='med:v2', models_param=models_param,  device='cuda:1')
norm = MedicationNormalizer(['ENT/DRUG','ENT/CLASS'], 'normalized_mention', ID='norm',romedi_path= romedi_path)

pipeline = [endlines,sections, sentenceSplitter, med, norm]

2020-11-05 15:02:05,023 loading file /export/home/cse180025/prod_information_extraction/data/models/apmed4/entities/final-model.pt
2020-11-05 15:02:09,402 loading file /export/home/cse180025/prod_information_extraction/data/models/apmed4/events/final-model.pt
2020-11-05 15:02:11,062 loading file /export/home/cse180025/prod_information_extraction/data/models/apmed4/drugblob/final-model.pt


In [4]:
data_path = pkg_resources.resource_filename('pymedext_eds', 'data/demo')
file_list = glob(data_path + '/*.txt')

docs = [rawtext_loader(x) for x in file_list]

In [5]:
for doc in docs:
    doc.annotate(pipeline)

Ignore 2 sentence(s) with no tokens.


In [10]:
pd.DataFrame.from_records(MedicationAnnotator.doc_to_omop(docs[1]))

,note_nlp_id,note_id,person_id,section_concept_id,snippet,offset_begin,offset_end,lexical_variant,note_nlp_concept_id,note_nlp_source_concept_id,nlp_system,nlp_date,nlp_datetime,term_exists,term_temporal,term_modifiers,validation_level_id
0,None,cas2,cas2,head,"Le 9 décembre 2010, une femme de 79 ans reçoit...",65,75,Pyostacine,J01FG01,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9963575005531311',snip...",automated
1,None,cas2,cas2,head,Un traitement par des soins locaux avec des p...,1254,1264,tulle gras,None,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9664367139339447'",automated
2,None,cas2,cas2,head,Un traitement par des soins locaux avec des p...,1294,1302,Plurexid,D08AC02,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9969384670257568',ATC_...",automated
3,None,cas2,cas2,head,"Le 17 février 2014, un traitement par Augment...",2362,2371,Augmentin,J01CR02,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9999349117279053',ATC_...",automated
4,None,cas2,cas2,head,"En raison d'une diarrhée, l’Augmentin® est re...",2583,2591,Solupred,H02AB06,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9998010993003845',ATC_...",automated
5,None,cas2,cas2,head,"En raison d'une diarrhée, l’Augmentin® est re...",2509,2518,Rocéphine,J01DD04,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9999445676803589',ENT/...",automated
6,None,cas2,cas2,head,"Le 28 février, de l'Oroken® est prescrit en a...",2687,2696,Pulmicort,R03BA02,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9989744424819946',ATC_...",automated
7,None,cas2,cas2,head,Sur la fiche d’admission remplie par le médec...,3549,3559,Pyostacine,J01FG01,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.8990024328231812',ATC_...",automated
8,None,cas2,cas2,head,Augmentin,3562,3571,Augmentin,J01CR02,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9985721111297607',ATC_...",automated
9,None,cas2,cas2,head,Après la réalisation de prélèvements bactério...,4041,4050,Rocéphine,J01DD04,ATC,medext_v3,2020-11-05,2020-11-05 15:04:55,None,None,"section='head',score='0.9982656836509705',ATC_...",automated
